<a href="https://colab.research.google.com/github/seti2000/image_recognition/blob/main/lost_and_found.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
!pip install tensorflow_hub matplotlib pillow

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
# Load a pre-trained model
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

# Create a simple tensor
tensor = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])

# Perform a basic operation
result = tf.reduce_mean(tensor)

print("Tensor:", tensor)
print("Mean:", result.numpy())

Tensor: tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
Mean: 3.5


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import io

# Load the pre-trained COCO SSD model
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

# Function to load and preprocess the image
def load_image(image_content):
    image = tf.image.decode_image(image_content, channels=3)
    return tf.image.convert_image_dtype(image, tf.float32)[tf.newaxis, ...]

# Function to run inference on an image
def run_inference(image):
    return model(image)

# Function to visualize the results
def visualize_results(image, output_dict, threshold=0.5):
    plt.figure(figsize=(12, 8))
    plt.imshow(image[0])

    boxes = output_dict['detection_boxes'][0].numpy()
    classes = output_dict['detection_classes'][0].numpy().astype(int)
    scores = output_dict['detection_scores'][0].numpy()

    height, width, _ = image[0].shape

    for i in range(min(boxes.shape[0], 10)):  # Display top 10 detections
        if scores[i] > threshold:
            ymin, xmin, ymax, xmax = boxes[i]
            (left, right, top, bottom) = (xmin * width, xmax * width,
                                          ymin * height, ymax * height)

            plt.gca().add_patch(plt.Rectangle((left, top), right - left, bottom - top,
                                              fill=False, edgecolor='red', linewidth=2))
            plt.text(left, top, f'{COCO_CLASSES[classes[i]-1]}: {scores[i]:.2f}',
                     bbox=dict(facecolor='yellow', alpha=0.5))

    plt.axis('off')

    # Save the plot to a buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close()

    return buf

# COCO class names
COCO_CLASSES = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter',
    'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra',
    'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
    'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
    'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
    'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]
COCO_TO_APP_CATEGORY = {
    'cell phone': 'Electronics',
    'laptop': 'Electronics',
    'tv': 'Electronics',
    'remote': 'Electronics',
    'keyboard': 'Electronics',
    'mouse': 'Electronics',
    'book': 'Books',
    'backpack': 'Others',
    'handbag': 'Others',
    'suitcase': 'Others',
    'tie': 'Clothes',
    'umbrella': 'Others',
    'bottle': 'Others',
    'cup': 'Others',
    'chair': 'Others',
    'couch': 'Others',
    'bed': 'Others',
    'dining table': 'Others',
    'clock': 'Others',
    'vase': 'Others',
    'scissors': 'Others',
    'teddy bear': 'Others',
    'hair drier': 'Electronics',
    'toothbrush': 'Others',
    # Add more mappings as needed
}

# For items not in the mapping, we'll categorize them as 'Others'
DEFAULT_CATEGORY = 'Others'
# Main function to process an image
def process_lost_and_found_image(image_content):
    image = load_image(image_content)
    output_dict = run_inference(image)
    visualization = visualize_results(image, output_dict)

    # Get detected objects
    classes = output_dict['detection_classes'][0].numpy().astype(int)
    scores = output_dict['detection_scores'][0].numpy()

    detected_objects = []
    app_categories = set()

    for i in range(len(scores)):
        if scores[i] > 0.5:  # Confidence threshold
            coco_class = COCO_CLASSES[classes[i]-1]
            detected_objects.append(coco_class)
            app_category = COCO_TO_APP_CATEGORY.get(coco_class, DEFAULT_CATEGORY)
            app_categories.add(app_category)

    return list(set(detected_objects)), list(app_categories), visualization


# This function would be called from your Flask route
def handle_image_upload(image_file):
    image_content = image_file.read()
    detected_objects, visualization = process_lost_and_found_image(image_content)
    return detected_objects, visualization